In [2]:
### ValDXer testing
import os


import pickle


In [3]:
import subprocess


In [4]:
MD_BPTI_dir = "/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/5PTI/BPTI_genvel3"


In [5]:
BPTI_dir = "/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/HDXer_tutorial/BPTI"
expt_dir = os.path.join(BPTI_dir, "BPTI_expt_data")

os.listdir(expt_dir)

segs_name = "BPTI_residue_segs.txt"
segs_path = os.path.join(expt_dir, segs_name)

hdx_name = "BPTI_expt_dfracs.dat"
hdx_path = os.path.join(expt_dir, hdx_name)
print(hdx_path)

rates_name = "BPTI_Intrinsic_rates.dat"
rates_path = os.path.join(expt_dir, rates_name)


/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/HDXer_tutorial/BPTI/BPTI_expt_data/BPTI_expt_dfracs.dat


In [6]:
sim_name = 'BPTI_MD'

sim_dir = os.path.join(MD_BPTI_dir)
print(sim_dir)

print(os.listdir(sim_dir))

md_reps = 5

rep_dirs = ["R_"+str(i+1) for i in range(md_reps)]

top_name = "APO_amber99_5PTI_1-nojump.pdb"

top_path = os.path.join(sim_dir, rep_dirs[0], top_name)

traj_name = "APO_amber99_5PTI_6-nojump.xtc"

# APO_amber99_5PTI_cat_{rep}.xtc

# this is the 

xtal_name= "5PTI_APO_amber99_H_solv_ions.gro"
xtal_path = os.path.join(sim_dir, rep_dirs[0], xtal_name)

traj_paths = [
    os.path.join(sim_dir, rep, f"APO_amber99_5PTI_cat_{idx+1}.xtc") for idx, rep in enumerate(rep_dirs)
]



print(top_path)
print(traj_paths)




/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/5PTI/BPTI_genvel3
['R_5', 'R_2', 'R_3', 'R_4', 'BPTI_HDX_bfactors.pdb', '5PTI_APO_amber99_H_solv_ions.gro', 'R_1']
/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/5PTI/BPTI_genvel3/R_1/APO_amber99_5PTI_1-nojump.pdb
['/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/5PTI/BPTI_genvel3/R_1/APO_amber99_5PTI_cat_1.xtc', '/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/5PTI/BPTI_genvel3/R_2/APO_amber99_5PTI_cat_2.xtc', '/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/5PTI/BPTI_genvel3/R_3/APO_amber99_5PTI_cat_3.xtc', '/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/5PTI/BPTI_genvel3/R_4/APO_amber99_5PTI_cat_4.xtc', 

In [7]:
import MDAnalysis as mda
import MDAnalysis
import numpy as np
import pandas as pd


In [8]:
# load xtal
u = mda.Universe(top_path)
print(len(u.residues))
# select first frame
# u = u.trajectory[0]



print(type(u))   
# print residues


58
<class 'MDAnalysis.core.universe.Universe'>


In [9]:
def calc_HDX(structure: mda.Universe)->float:
    """This functions calculates the HDX-MS free energy using the Best Vendruscolo method
    For each residue this counts up the number of heavy atoms from the amide nitogren and 
    hydrogen bonds made with the amide hydrogen. The number of contacts are multiplied by
    their beta weights and then summed up to give the free energy of the protein.    

    """
    B_C, B_H = 0.35, 2.0
    cut_C, cut_H = 6.5, 2.4 # angstroms as in MDA

    n_C, n_H = 0, 0

    for res in structure.residues:
        resid = res.resid
        print("Resi: ", resid)

        amide_N = res.atoms.select_atoms("name N")
        amide_H = res.atoms.select_atoms("name H or name H1 or name H2 or name H3")
        # print(amide_N.atoms)
        amide_N_pos_string = " ".join([str(i) for i in amide_N.positions[0]])
        # print(amide_N_pos_string)

        # find heavy atoms within cut-off of amide nitrogen in universe
        heavy_atom_selection = f"point {amide_N_pos_string} {cut_C} and not name H* and not (name N and resid {resid})"
        heavy_atoms = structure.select_atoms(heavy_atom_selection)
        # print(len(heavy_atoms))
        n_C += len(heavy_atoms)

        amide_H_positions = amide_H.positions
        # repeat for acceptor atoms for all amide hydrogens
        for hydrogen in amide_H_positions:
            hydrogen_pos_string = " ".join([str(i) for i in hydrogen])
            # print(hydrogen_pos_string)
            acceptor_atom_selection = f"point {hydrogen_pos_string} {cut_H} and (type O or type N or type S) and not (name N and resid {resid})"
            acceptor_atoms = structure.select_atoms(acceptor_atom_selection)
            # print(len(acceptor_atoms))
            n_H += len(acceptor_atoms)

    HDX_free_energy = B_C * n_C + B_H * n_H

    return HDX_free_energy

    
        
print(calc_HDX(u))

Resi:  1
Resi:  2
Resi:  3
Resi:  4
Resi:  5
Resi:  6
Resi:  7
Resi:  8
Resi:  9
Resi:  10
Resi:  11
Resi:  12
Resi:  13
Resi:  14
Resi:  15
Resi:  16
Resi:  17
Resi:  18
Resi:  19
Resi:  20
Resi:  21
Resi:  22
Resi:  23
Resi:  24
Resi:  25
Resi:  26
Resi:  27
Resi:  28
Resi:  29
Resi:  30
Resi:  31
Resi:  32
Resi:  33
Resi:  34
Resi:  35
Resi:  36
Resi:  37
Resi:  38
Resi:  39
Resi:  40
Resi:  41
Resi:  42
Resi:  43
Resi:  44
Resi:  45
Resi:  46
Resi:  47
Resi:  48
Resi:  49
Resi:  50
Resi:  51
Resi:  52
Resi:  53
Resi:  54
Resi:  55
Resi:  56
Resi:  57
Resi:  58
1029.85


In [10]:
top = mda.Universe(top_path).select_atoms("protein")
print(top.atoms)


<AtomGroup [<Atom 1: N of type N of resname ARG, resid 1 and segid SYSTEM and altLoc >, <Atom 2: H1 of type H of resname ARG, resid 1 and segid SYSTEM and altLoc >, <Atom 3: H2 of type H of resname ARG, resid 1 and segid SYSTEM and altLoc >, ..., <Atom 890: C of type C of resname ALA, resid 58 and segid SYSTEM and altLoc >, <Atom 891: OC1 of type O of resname ALA, resid 58 and segid SYSTEM and altLoc >, <Atom 892: OC2 of type O of resname ALA, resid 58 and segid SYSTEM and altLoc >]>


In [11]:
top = mda.Universe(top_path).select_atoms("protein")
# select protein

print(calc_HDX(top))


Resi:  1
Resi:  2
Resi:  3
Resi:  4
Resi:  5
Resi:  6
Resi:  7
Resi:  8
Resi:  9
Resi:  10
Resi:  11
Resi:  12
Resi:  13
Resi:  14
Resi:  15
Resi:  16
Resi:  17
Resi:  18
Resi:  19
Resi:  20
Resi:  21
Resi:  22
Resi:  23
Resi:  24
Resi:  25
Resi:  26
Resi:  27
Resi:  28
Resi:  29
Resi:  30
Resi:  31
Resi:  32
Resi:  33
Resi:  34
Resi:  35
Resi:  36
Resi:  37
Resi:  38
Resi:  39
Resi:  40
Resi:  41
Resi:  42
Resi:  43
Resi:  44
Resi:  45
Resi:  46
Resi:  47
Resi:  48
Resi:  49
Resi:  50
Resi:  51
Resi:  52
Resi:  53
Resi:  54
Resi:  55
Resi:  56
Resi:  57
Resi:  58
1029.85


In [12]:
trajs = mda.Universe(top_path, traj_paths)
# select protein
print(len(trajs.trajectory))
# calc HDX over trajectory
stride = 100

traj_HDX = []
for idx, ts in enumerate(trajs.trajectory):
    if idx % stride == 0:
        print(idx)
        traj_HDX.append(calc_HDX(trajs))
    
    # print(calc_HDX(trajs))
    # break

51135
0
Resi:  1
Resi:  2
Resi:  3
Resi:  4
Resi:  5
Resi:  6
Resi:  7
Resi:  8
Resi:  9
Resi:  10
Resi:  11
Resi:  12
Resi:  13
Resi:  14
Resi:  15
Resi:  16
Resi:  17
Resi:  18
Resi:  19
Resi:  20
Resi:  21
Resi:  22
Resi:  23
Resi:  24
Resi:  25
Resi:  26
Resi:  27
Resi:  28
Resi:  29
Resi:  30
Resi:  31
Resi:  32
Resi:  33
Resi:  34
Resi:  35
Resi:  36
Resi:  37
Resi:  38
Resi:  39
Resi:  40
Resi:  41
Resi:  42
Resi:  43
Resi:  44
Resi:  45
Resi:  46
Resi:  47
Resi:  48
Resi:  49
Resi:  50
Resi:  51
Resi:  52
Resi:  53
Resi:  54
Resi:  55
Resi:  56
Resi:  57
Resi:  58
100
Resi:  1
Resi:  2
Resi:  3
Resi:  4
Resi:  5
Resi:  6
Resi:  7
Resi:  8
Resi:  9
Resi:  10
Resi:  11
Resi:  12
Resi:  13
Resi:  14
Resi:  15
Resi:  16
Resi:  17
Resi:  18
Resi:  19
Resi:  20
Resi:  21
Resi:  22
Resi:  23
Resi:  24
Resi:  25
Resi:  26
Resi:  27
Resi:  28
Resi:  29
Resi:  30
Resi:  31
Resi:  32
Resi:  33
Resi:  34
Resi:  35
Resi:  36
Resi:  37
Resi:  38
Resi:  39
Resi:  40
Resi:  41
Resi:  42
Resi: 

In [13]:
# plot HDX over trajectory using plotly
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(y=traj_HDX))
fig.show()

In [14]:
# plot HDX as histogram using plotly bins = 100
fig = go.Figure(data=[go.Histogram(x=traj_HDX, nbinsx=200)])
fig.show()



In [15]:
xtal = mda.Universe(xtal_path)

# select protein
xtal = xtal.select_atoms("protein")
# print(xtal.atoms)

xtal_HDX= calc_HDX(xtal)

Resi:  1
Resi:  2
Resi:  3
Resi:  4
Resi:  5
Resi:  6
Resi:  7
Resi:  8
Resi:  9
Resi:  10
Resi:  11
Resi:  12
Resi:  13
Resi:  14
Resi:  15
Resi:  16
Resi:  17
Resi:  18
Resi:  19
Resi:  20
Resi:  21
Resi:  22
Resi:  23
Resi:  24
Resi:  25
Resi:  26
Resi:  27
Resi:  28
Resi:  29
Resi:  30
Resi:  31
Resi:  32
Resi:  33
Resi:  34
Resi:  35
Resi:  36
Resi:  37
Resi:  38
Resi:  39
Resi:  40
Resi:  41
Resi:  42
Resi:  43
Resi:  44
Resi:  45
Resi:  46
Resi:  47
Resi:  48
Resi:  49
Resi:  50
Resi:  51
Resi:  52
Resi:  53
Resi:  54
Resi:  55
Resi:  56
Resi:  57
Resi:  58


In [16]:
# plot HDX over trajectory using plotly dotted line at xtal HDX
fig = go.Figure(data=go.Scatter(y=traj_HDX))
fig.add_trace(go.Scatter(y=[xtal_HDX]*len(traj_HDX), mode='lines', name='xtal'))
# add title
fig.update_layout(title='HDX FE over 500ns MD trajectory')
# add xaxis labels
fig.update_xaxes(title_text='Frame')
# add yaxis labels
fig.update_yaxes(title_text='HDX FE /kcal mol-1')

fig.show()


In [121]:
import numpy as np

def calc_HDX_by_res(structure: mda.Universe, B_C=0.35, B_H=2.0, cut_C=6.5, cut_H=2.4):
    # cut_C, cut_H = 6.5, 2.4  # Angstroms

    n_C, n_H = [], []

    for res in structure.residues:
        resid = res.resid
        # print("Resi: ", resid)

        amide_N = res.atoms.select_atoms("name N")
        amide_H = res.atoms.select_atoms("name H or name H1 or name H2 or name H3")

        amide_N_pos_string = " ".join([str(i) for i in amide_N.positions[0]])

        heavy_atom_selection = f"point {amide_N_pos_string} {cut_C} and not name H* and not (name N and resid {resid})"
        heavy_atoms = structure.select_atoms(heavy_atom_selection)
        n_C.append(len(heavy_atoms))
        
        amide_H_positions = amide_H.positions
        total = 0
        for hydrogen in amide_H_positions:
            hydrogen_pos_string = " ".join([str(i) for i in hydrogen])
            acceptor_atom_selection = f"point {hydrogen_pos_string} {cut_H} and (type O or type N or type S) and not (name N and resid {resid})"
            acceptor_atoms = structure.select_atoms(acceptor_atom_selection)

            total += len(acceptor_atoms)

        # Handle case where no hydrogens are found
        if len(amide_H_positions) > 0:
            n_H.append(total / len(amide_H_positions))
        else:
            n_H.append(0)  # Or set a default value or skip the calculation

    n_C = np.array(n_C)
    n_H = np.array(n_H)

    HDX_FE_C = B_C * n_C
    HDX_FE_H = B_H * n_H
    HDX_FE = HDX_FE_C + HDX_FE_H  # Sum for each residue

    return HDX_FE_C, HDX_FE_H

# Example usage
# hdx_bfactors = calc_HDX_by_res(u)


In [98]:
# import MDAnalysis as mda
# from MDAnalysis.analysis import align
# ### THIS CANT 
# # Load the trajectory and topology
# trajs = mda.Universe(top_path, traj_paths)

# # Select the reference (e.g., the backbone of the first frame)
# ref = trajs.select_atoms('backbone')  # You can change 'backbone' to your selection

# # Initialize the AlignTraj class with your universe, the reference universe, and the selection of atoms to be aligned
# alignment = align.AlignTraj(trajs, trajs, select='backbone', in_memory=True)  # Change 'backbone' to your selection if needed

# # Run the alignment
# alignment.run()

# print(len(trajs.trajectory))

# # Calculate HDX over trajectory
# stride = 100
# hdx_bfactor_pdb_name = "BPTI_HDX_bfactors.pdb"
# hdx_bfactor_pdb_path = os.path.join(sim_dir, hdx_bfactor_pdb_name)
# print(hdx_bfactor_pdb_path)

# with mda.Writer(hdx_bfactor_pdb_path, multiframe=True) as W:
#     hdx_tot = 0
#     for idx, ts in enumerate(trajs.trajectory):
#         if idx % stride == 0:
#             hdx_frame = calc_HDX_by_res(trajs)
#             # Add bfactor to structure for each residue
#             hdx_tot += hdx_frame
#     hdx_avg = hdx_tot / (len(trajs.trajectory)/stride)

#     # Add bfactor to structure for each residue
#     for idx, res in enumerate(trajs.residues):
#         res.atoms.bfactors = hdx_avg[idx]
#     W.write(trajs)



51135
/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/5PTI/BPTI_genvel3/BPTI_HDX_bfactors.pdb
Resi:  1
Resi:  2
Resi:  3
Resi:  4
Resi:  5
Resi:  6
Resi:  7
Resi:  8
Resi:  9
Resi:  10
Resi:  11
Resi:  12
Resi:  13
Resi:  14
Resi:  15
Resi:  16
Resi:  17
Resi:  18
Resi:  19
Resi:  20
Resi:  21
Resi:  22
Resi:  23
Resi:  24
Resi:  25
Resi:  26
Resi:  27
Resi:  28
Resi:  29
Resi:  30
Resi:  31
Resi:  32
Resi:  33
Resi:  34
Resi:  35
Resi:  36
Resi:  37
Resi:  38
Resi:  39
Resi:  40
Resi:  41
Resi:  42
Resi:  43
Resi:  44
Resi:  45
Resi:  46
Resi:  47
Resi:  48
Resi:  49
Resi:  50
Resi:  51
Resi:  52
Resi:  53
Resi:  54
Resi:  55
Resi:  56
Resi:  57
Resi:  58


TypeError: unsupported operand type(s) for +=: 'int' and 'tuple'

In [122]:
def calc_HDX_beta(structure: mda.Universe, B_C=0.35, B_H=2.0):
    """This functions calculates the HDX-MS free energy using the Best Vendruscolo method
    For each residue this counts up the number of heavy atoms from the amide nitogren and 
    hydrogen bonds made with the amide hydrogen. The number of contacts are multiplied by
    their beta weights and then summed up to give the free energy of the protein.    

    """
    cut_C, cut_H = 6.5, 2.4 # angstroms as in MDA

    n_C, n_H = 0, 0

    for res in structure.residues:
        resid = res.resid
        print("Resi: ", resid)

        amide_N = res.atoms.select_atoms("name N")
        amide_H = res.atoms.select_atoms("name H or name H1 or name H2 or name H3")
        # print(amide_N.atoms)
        amide_N_pos_string = " ".join([str(i) for i in amide_N.positions[0]])
        # print(amide_N_pos_string)

        # find heavy atoms within cut-off of amide nitrogen in universe
        heavy_atom_selection = f"point {amide_N_pos_string} {cut_C} and not name H* and not (name N and resid {resid})"
        heavy_atoms = structure.select_atoms(heavy_atom_selection)
        # print(len(heavy_atoms))
        n_C += len(heavy_atoms)

        amide_H_positions = amide_H.positions
        # repeat for acceptor atoms for all amide hydrogens
        for hydrogen in amide_H_positions:
            hydrogen_pos_string = " ".join([str(i) for i in hydrogen])
            # print(hydrogen_pos_string)
            acceptor_atom_selection = f"point {hydrogen_pos_string} {cut_H} and (type O or type N or type S) and not (name N and resid {resid})"
            acceptor_atoms = structure.select_atoms(acceptor_atom_selection)
            # print(len(acceptor_atoms))
            n_H += len(acceptor_atoms)


    return  B_C * n_C, B_H * n_H

    

In [201]:
def calc_total_HDX_beta(universe:mda.Universe, B_C, B_H, stride=100,segs:list=[None]):
    seg_indices = [s-1 for s in segs]
    HDX_free_energy = 0
    for ts in universe.trajectory[::stride]:
        HDX_FE_C, HDX_FE_H = calc_HDX_by_res(universe, B_C, B_H)
        HDX_FE_C, HDX_FE_H = HDX_FE_C[seg_indices], HDX_FE_H[seg_indices]
        HDX_free_energy += HDX_FE_C + HDX_FE_H
    return HDX_free_energy/(len(universe.trajectory)/stride)

from ValDX.helpful_funcs import segs_to_df, dfracs_to_df
from ValDX.VDX_Settings import Settings
settings = Settings()
times = settings.times


def load_HDX_data(pf_path, segs_txt_path, names=times):

    pf_df = pd.read_csv(pf_path, sep='\s+', skiprows=[0], header=None)


    segs_df = pd.read_csv(segs_txt_path, sep='\s+', header=None)

    # segs is a list of tuples from the columns 0 and 1 segs_df 
    segs = segs_df[[0,1]].to_numpy().flatten().tolist()
    segs = sorted(set(segs))
    # select column 0 based on segs
    pf_df = pf_df.loc[pf_df[0].isin(segs)]
    pf_segs = pf_df[1].to_list()

    # set segs to pf segs
    segs = pf_df[0].to_list()
    
    return pf_segs, segs

In [202]:
pf_path ="/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/HDXer_tutorial/BPTI/BPTI_expt_data/BPTI_expt_PFs.dat"
pf_df = pd.read_csv(pf_path, sep='\s+', skiprows=[0], header=None)
pf_df.head()    
# pfs = pf_df[1].to_list()
# print(pfs)

,0,1
0,5,3.704
1,6,3.229
2,7,3.574
3,10,2.500
4,12,1.668


In [203]:
segs_path = "/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/data/BPTI_500ns/train_MD_Simulated_1/train_residue_segs__MD_Simulated_.txt"
segs_df = pd.read_csv(segs_path, sep='\s+', header=None)

# segs is a list of tuples from the columns 0 and 1 segs_df 
segs = segs_df[[0,1]].to_numpy().flatten().tolist()
segs = sorted(set(segs))

print(segs)
# create set of segs


# segs_df.head()

[4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56]


In [204]:
pf_df = pf_df.loc[pf_df[0].isin(segs)]

pf_df

,0,1
0,5,3.704
1,6,3.229
2,7,3.574
3,10,2.500
4,12,1.668
5,14,2.400
6,16,3.024
7,17,0.761
8,18,5.375
9,19,0.980


In [205]:

def HDX_sim_experiment_MAE(hdx_pfs, segs, universe,
                            B_C=0.35, B_H=2.0,
                           stride=100):
    """This function calculates the MAE between the experimental and simulated HDX-MS data
    for a given simulation. The simulation must be in the form of a MDAnalysis Universe
    object. The experimental data is in the form of a lists with segs being residue indices
    """
    # select segs (resids) from universe

    # calc total HDX from simulation
    sim_HDX_free_energy = calc_total_HDX_beta(universe, B_C, B_H, stride, segs)
    # print(sim_HDX_free_energy)

    # calc total HDX free energy from experiment PFs using dG = RTln(PF)
    R = 8.314
    T = 298
    expt_HDX_free_energy = 0
    for pf in hdx_pfs:
        expt_HDX_free_energy += (pf)
    sum_pfs = sum(hdx_pfs)
    # print(sum_pfs)
    # print(expt_HDX_free_energy)
        
    # calc MAE
    MAE = np.abs(sim_HDX_free_energy - expt_HDX_free_energy)
    # print(MAE)
    # print(len(MAE))
    return MAE.mean()




In [206]:
# bayesian optmisation of beta parameters

from bayes_opt import BayesianOptimization

# define black box function 

def black_box_optimiser(B_C, B_H):
    
    # define universe
    # xtal = mda.Universe(xtal_path)
    trajs = mda.Universe(top_path, traj_paths[0])
    
    segs_path = "/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/data/BPTI_500ns/train_MD_Simulated_1/train_residue_segs__MD_Simulated_.txt"
    pf_path ="/Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/ValDX/raw_data/HDXer_tutorial/BPTI/BPTI_expt_data/BPTI_expt_PFs.dat"
    hdx_pfs, segs = load_HDX_data(pf_path, segs_path)

    # calc MAE
    MAE = HDX_sim_experiment_MAE(hdx_pfs, segs, trajs, B_C, B_H)
    print(MAE)
    return -MAE


In [207]:
MAE = black_box_optimiser(0.35, 2.0)
print(MAE)

138.97214324110976
-138.97214324110976


In [208]:
pbounds = {'B_C': (0.1, 1), 'B_H': (1, 10)}

optimizer = BayesianOptimization(
    f=black_box_optimiser,
    pbounds=pbounds,
    random_state=1,
    allow_duplicate_points=True
)

In [209]:
optimizer.maximize(
    init_points=2,
    n_iter=125,
)

|   iter    |  target   |    B_C    |    B_H    |
-------------------------------------------------
128.29733648879437
| 1         | -128.3    | 0.4753    | 7.483     |
150.4231075256997
| 2         | -150.4    | 0.1001    | 3.721     |
136.60039967530054
| 3         | -136.6    | 0.3115    | 7.587     |
118.09742736986568
| 4         | -118.1    | 0.6766    | 7.355     |
127.95415745243989
| 5         | -128.0    | 0.482     | 7.484     |
101.35103919965451
| 6         | -101.4    | 1.0       | 7.606     |
100.68763633801285
| 7         | -100.7    | 1.0       | 8.459     |
99.68357116168535
| 8         | -99.68    | 1.0       | 9.751     |
145.54811263823404
| 9         | -145.5    | 0.1       | 10.0      |
100.90737971784615
| 10        | -100.9    | 0.986     | 9.1       |
106.48537340421223
| 11        | -106.5    | 1.0       | 1.0       |
152.54322600385876
| 12        | -152.5    | 0.1       | 1.0       |
102.6980573874274
| 13        | -102.7    | 1.0       | 5.873     |
148.90

KeyboardInterrupt: 